# Ramsey King
# DSC 550 - Data Mining
# September 11, 2021
# Week 2: Handling Categorical Data, Text, Dates & Times
# Exercise 2.2 


You can find the dataset controversial-comments.jsonl for this exercise in the Weekly Resources: Week 2 Data Files.
Pre-processing Text: For this part, you will start by reading the controversial-comments.jsonl file into a DataFrame.

In [6]:
import pandas as pd
import numpy as np

comments_df = pd.read_json('controversial-comments.jsonl', lines=True)
comments_df.head()



,con,txt
0,0,Well it's great that he did something about th...
1,0,You are right Mr. President.
2,0,You have given no input apart from saying I am...
3,0,I get the frustration but the reason they want...
4,0,I am far from an expert on TPP and I would ten...


A. Convert all text to lowercase letters.

In [9]:
comments_df['txt'] = comments_df['txt'].apply(str.lower)
comments_df['txt'][:5]

0    well it's great that he did something about th...
1                         you are right mr. president.
2    you have given no input apart from saying i am...
3    i get the frustration but the reason they want...
4    i am far from an expert on tpp and i would ten...
Name: txt, dtype: object

B. Remove all punctuation from the text.


In [15]:
import re
comments_df['txt'] = comments_df['txt'].apply(lambda remove_punct: re.sub(r'[^\w\s]', '', remove_punct))
comments_df['txt'][:5]

0    well its great that he did something about tho...
1                           you are right mr president
2    you have given no input apart from saying i am...
3    i get the frustration but the reason they want...
4    i am far from an expert on tpp and i would ten...
Name: txt, dtype: object

C. Remove stop words.


In [41]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words = stopwords.words('english')

def rem_stop_words(rem_list):
    return [word for word in rem_list if word not in stop_words]


'''
# THIS CODE WAS USED TO MAKE SURE THAT THE FUNCTIONS WORKED BEFORE RUNNING IT ON THE ENTIRE DATAFRAME
comments_df_test = comments_df[:10].copy()
comments_df_test['txt_tokenized'] = comments_df_test['txt'].apply(word_tokenize)
print(comments_df_test['txt_tokenized'][:5])
comments_df_test['txt_tokenized'] = comments_df_test['txt_tokenized'].apply(rem_stop_words)
comments_df_test['txt_tokenized'][:5]
'''
comments_df['txt_tokenized'] = comments_df['txt'].apply(word_tokenize)
comments_df['txt_tokenized'] = comments_df['txt_tokenized'].apply(rem_stop_words)
comments_df['txt_tokenized'][:5]

0    [well, great, something, beliefs, office, doub...
1                               [right, mr, president]
2    [given, input, apart, saying, wrong, argument,...
3    [get, frustration, reason, want, way, foundati...
4    [far, expert, tpp, would, tend, agree, lot, pr...
Name: txt_tokenized, dtype: object

D. Apply NLTK’s PorterStemmer.

In [42]:
comments_df.head()

,con,txt,txt_tokenized
0,0,well its great that he did something about tho...,"[well, great, something, beliefs, office, doub..."
1,0,you are right mr president,"[right, mr, president]"
2,0,you have given no input apart from saying i am...,"[given, input, apart, saying, wrong, argument,..."
3,0,i get the frustration but the reason they want...,"[get, frustration, reason, want, way, foundati..."
4,0,i am far from an expert on tpp and i would ten...,"[far, expert, tpp, would, tend, agree, lot, pr..."
